<a href="https://colab.research.google.com/github/schilloji/ml_fractal_3_assignment/blob/build/problem_3_task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
import os
import shutil
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
drive.mount('/content/drive')

In [3]:
pretain_path='/content/drive/MyDrive/data/charts/alexnet/'
label_path = '/content/drive/MyDrive/data/charts/'
lbl_df=pd.read_csv(label_path+str('train_val.csv'))

'''
I am using Pytorch Implementation for building the alexnet pretraining model. To prepare the data, we need to have the images into the respective
folders of classes. Since we have one folder for all the images and labels in excel file, I will creating 5 folders for all the 5 classe and moving all the resprective images
to the those folders.

The below code creates the /alexnet folder inside the /charts folder and inside the /alexnet folder we creates the below folders
/dot_line
/pie
/hbar_categorical
/line
/vbar_categorical
'''
try:
  os.mkdir(label_path+str('alexnet'))
  for i in lbl_df['type'].unique():  
    class_path=pretain_path+str(i)
    if os.path.exists(class_path):
      pass
    else:
      os.mkdir(class_path)
except OSError as e:
  print(f"No need to create {e}")

image_path = '/content/drive/MyDrive/data/charts/train_val'


'''
The below code helps in moving the respective images from the /train_val folder to the /alexnet resprective class folder.
'''
for file in os.listdir(image_path):
  indx=int(file.split('.')[0])
  file_path = os.path.join(image_path, file)
  class_=lbl_df.iloc[indx,:][1]
  target_path=os.path.join(pretain_path,class_)
  if os.path.exists(os.path.join(pretain_path,class_,file)):
    pass
  else:
    if class_=='dot_line':
      shutil.copy(file_path, target_path)
    if class_=='pie':
      shutil.copy(file_path, target_path)
    if class_=='hbar_categorical':
      shutil.copy(file_path, target_path)
    if class_=='line':
      shutil.copy(file_path, target_path)
    if class_=='vbar_categorical':
      shutil.copy(file_path, target_path)



No need to create [Errno 17] File exists: '/content/drive/MyDrive/data/charts/alexnet'


In [ ]:
# Define transforms for the training and validation sets
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


pretain_path='/content/drive/MyDrive/charts/alexnet/'
train_dataset = ImageFolder(root=pretain_path, transform=train_transforms)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model = models.alexnet(pretrained=True)
model.eval()


In [ ]:
num_classes = 5
model.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes)
model.eval()

In [ ]:
model=model.to(device)
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        # print(inputs)
        # print(labels)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy on the validation set: {:.2f}%'.format(correct / total * 100))